In [1]:
# https://archive.ics.uci.edu/ml/datasets/glass+identification

# Glass 비교

# Import Module

In [2]:
from __future__ import print_function
import numpy as np
import pandas as pd
# import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp 
import statsmodels.api as sm 

import warnings
warnings.filterwarnings(action='ignore')
sns.set()

# from keras.models import Model
from sklearn.linear_model import LogisticRegression #--------------- 모델 (회귀모형 x에 따라 y값이 바뀜)
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score 
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
mm = MinMaxScaler()
rb = RobustScaler()
ss = StandardScaler()

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from  sklearn.ensemble import GradientBoostingClassifier

# 데이터 가져오기

In [3]:
glass = pd.read_csv("./dataset/glass.csv")

In [4]:
# 가져온 데이터 문제(X)/답안지(y)로 나누기
X = glass.iloc[:,:-1]  #  [행,열]     # iloc 설명 : https://blog.naver.com/book_endlesspr/222733692151
y = glass.iloc[:,-1]   #  [행,열]
print(X.shape, y.shape)

(214, 9) (214,)


# 데이터 정보확인

In [5]:
#Attribute Information:

# 1. Id number: 1 to 214
# 2. RI: refractive index
# 3. Na: Sodium (unit measurement: weight percent in corresponding oxide, as are attributes 4-10)
# 4. Mg: Magnesium
# 5. Al: Aluminum
# 6. Si: Silicon
# 7. K: Potassium
# 8. Ca: Calcium
# 9. Ba: Barium
# 10. Fe: Iron
# 11. Type of glass: (class attribute)
# -- 1 building_windows_float_processed
# -- 2 building_windows_non_float_processed
# -- 3 vehicle_windows_float_processed
# -- 4 vehicle_windows_non_float_processed (none in this database)
# -- 5 containers
# -- 6 tableware
# -- 7 headlamps

## DataFrame 구조확인

In [6]:
glass.keys()

Index(['ri', 'na', 'mg', 'al', 'si', 'k', 'ca', 'ba', 'fe', 'glass_type'], dtype='object')

In [1]:
glass.head()

NameError: name 'glass' is not defined

In [8]:
print(glass.shape)
print(glass.info())

(214, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ri          214 non-null    float64
 1   na          214 non-null    float64
 2   mg          214 non-null    float64
 3   al          214 non-null    float64
 4   si          214 non-null    float64
 5   k           184 non-null    float64
 6   ca          214 non-null    float64
 7   ba          214 non-null    float64
 8   fe          213 non-null    float64
 9   glass_type  214 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 16.8 KB
None


## 결측치

* 피쳐 k  : 결측 '0'으로 채우기
* 피쳐 fe : 결측 '평균값'으로 채우기, 평균값은 반올림해 소수점2자리까지만 사용
* 반올림 : round(), 올림 : math.ceil(), 내림:math.floor(), 소수점 버림:math.trunc() 

In [9]:
# glass.isna().sum()
glass.isna().sum()[glass.isna().sum() > 0]

k     30
fe     1
dtype: int64

* 피쳐 k : 결측 '0'으로 채우기

In [10]:
glass['k'] = glass['k'].fillna(0)
# glass['k'].fillna(0, inplace=True)

In [11]:
glass.isna().sum()[glass.isna().sum() > 0]

fe    1
dtype: int64

* 피쳐 fe : 결측  glass_type별 fe '평균값'으로 채우기, 평균값은 반올림해 소수점2자리까지만 사용 

In [12]:
# glass_type 별 fe 평균값
a = glass.groupby("glass_type")["fe"].transform("mean")

# glass_type 별 fe 평균값 + 소수점 둘째자리에서 반올림
b = round(glass.groupby("glass_type")["fe"].transform("mean"),2)

In [13]:
# glass_type 별 fe 평균값 + 소수점 둘째자리에서 반올림 으로 결측 채우기
glass["fe"] = glass["fe"].fillna( round(glass.groupby("glass_type")["fe"].transform("mean"),2) )

In [14]:
glass.isna().sum()

ri            0
na            0
mg            0
al            0
si            0
k             0
ca            0
ba            0
fe            0
glass_type    0
dtype: int64

## 신규피쳐 만들기
<pre>
* 'household' 파생피쳐 추가
*  Window 용 glass type =  0
*  그 외( Household 용)  glass type = 0  

11. Type of glass: (class attribute)
-- 1 building_windows_float_processed
-- 2 building_windows_non_float_processed
-- 3 vehicle_windows_float_processed
-- 4 vehicle_windows_non_float_processed (none in this database)
-- 5 containers
-- 6 tableware
-- 7 headlamps

In [15]:
glass['household'] = glass['glass_type'].map({ 1:0 , 2:0 , 3:0 , 4:0 , 5:1 , 6:1 , 7:1 })
glass.head()

,ri,na,mg,al,si,k,ca,ba,fe,glass_type,household
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1,0
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1,0
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1,0
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1,0
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1,0


# 모델 학습 및 평가

## 모델학습 
* train_test 분리
* test 데이터 크기는 전체 데이터의 20% 사용

In [16]:
X = glass.drop(['glass_type'],  axis=1)
y = glass['glass_type']
# X = glass.iloc[: , :-1]
# y = glass['glass_type']

print(X.shape, y.shape)

(214, 10) (214,)


In [17]:
train_X,test_X,train_y,test_y = train_test_split(X, y, test_size=0.2,random_state=1111, shuffle=True) 
print(train_X.shape , test_X.shape,  train_y.shape,test_y.shape)

(171, 10) (43, 10) (171,) (43,)


## 모델평가

<pre>
* 하이퍼파라미터 : 모델링할 때 사용자가 직접 세팅해주는 값
* 파라미터 : 매개변수 
* fit : 학습
* predict : 예측
* model : 학습모델
<b>
* model.fit(학습문제, 학습답안)
* model.predict(시험문제)

- 분류모델 : LogisticRegression (기본 파라미터 사용)
- 점수 : f1

In [18]:
lr_model = LogisticRegression(penalty='l2', C=1.0)

lr_model.fit(train_X, train_y)
lr_pred = lr_model.predict(test_X)

# print(f'intercept   = {lr_model.intercept_}')   # 편향 / 절편 / 파라미터
# print(f'coefficient = {lr_model.coef_}')        # 기울기 / 가중치 / 계수 
# y = ax + b ( a = coefficient / b = intercept )
# https://blog.naver.com/studyseony/222714186671

In [19]:
print('Confusion matrix')    # https://nittaku.tistory.com/294
print(confusion_matrix(test_y, lr_pred))

print(f"잘못 분류된 샘플 개수: {(test_y != lr_pred).sum()}")
print(f"Accuracy (정확도)  : {accuracy_score(test_y, lr_pred):.4f}")
print(f"Precision (정밀도) : {precision_score(test_y, lr_pred, average='macro'):.4f}")
print(f"Recall (재현율)    : {recall_score(test_y, lr_pred, average='macro'):.4f}")
print(f"F1                 : {f1_score(test_y, lr_pred, average='macro'):.4f}")

Confusion matrix
[[11  4  0  0  0  0]
 [ 6  9  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  2  0  2  0  1]
 [ 0  0  0  0  2  0]
 [ 0  0  0  0  0  5]]
잘못 분류된 샘플 개수: 14
Accuracy (정확도)  : 0.6744
Precision (정밀도) : 0.6741
Recall (재현율)    : 0.6222
F1                 : 0.6245


# 교차검증
- Cross Validation


< 사용하는 이유 >
- 모델의 변동성을 줄여준다.
- over fitting 문제 해결


< 사용방법 >
1. train / test 분리
2. 모델별 성능 기록
3. 매 단계 마다 다른 partition 으로 1,2번 수행
4. 모델의 최종성능은 매 단계의 test 성능의 평균을 계산

In [20]:
# KFold()           : 무작위 증강
# StratifiedKFold() : target을 train,test에 골로루 증강
# cross_val_score() : 증강해서 점수만 리턴
# GridSearchCV      : 증강 + 모델튜닝

from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, GridSearchCV
kf = KFold()
skf = StratifiedKFold()
# cvs = cross_val_score()
# gcv = GridSearchCV()

## KFold (회귀)

In [21]:
# split=5면 전체를 5개로 나누고 돌아가면서 test로 지정 후 검사함. 그 점수들을 평균내서 모델의 평가성능 도출 (적은 데이터로 여러번 학습하는 효과)
# 200개 데이터를 8:2로 나누면 40문제를 풀지만 fold는 200문제 푼다. (5배의 학습)

In [23]:
kf = KFold(n_splits=5, shuffle=True, random_state=2022)       # 샘플이 어느 한쪽에 치우치지 않도록 Shuffle = True (예외. 시계열은 무조건  shuffle=False)
cm_list = []      # Confusion Matrix 용
f1_list = []      # F1 용

for train_idx, test_idx in kf.split(X):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]       # iloc : train 데이터셋에서 인덱스가 train_idx와 동일한 것만 추출
    X_test,  y_test  = X.iloc[test_idx], y.iloc[test_idx]         # iloc : test 데이터셋에서 인덱스가 test_idx와 동일한 것만 추출
    print("TRAIN:", train_idx)
    print("TEST:", test_idx)
    
    lr_model = LogisticRegression(penalty='l2', C=1.0)            # LogisticRegression  => Regressor 라고 부르지만 분류모델임 !
    lr_model.fit(X_train, y_train)                                # penalty = 'l2' 는 규제 (https://teddylee777.github.io/scikit-learn/scikit-learn-linear-with-regularizations)
    lr_pred = lr_model.predict(X_test)
    print('Confusion matrix')
    print(confusion_matrix(y_test, lr_pred))   
    cm_list.append(confusion_matrix(y_test, lr_pred)) 
    print(f"F1        : {f1_score(y_test, lr_pred, average='macro'):.4f}")
    f1_list.append(f1_score(y_test, lr_pred, average='macro'))
print('fl score : ',np.mean(f1_list))

TRAIN: [  0   1   2   3   4   5   6   7   9  11  12  13  14  15  16  17  18  19
  21  23  24  25  26  27  28  29  30  31  33  34  35  37  39  40  41  42
  44  45  46  47  48  50  51  52  53  54  55  56  58  60  61  62  63  64
  65  66  67  68  69  70  71  72  75  76  77  78  79  80  81  82  83  84
  86  87  88  90  91  92  93  94  95  96  97  98  99 100 101 103 104 106
 107 108 109 110 112 113 114 115 117 119 121 122 125 126 128 129 130 131
 132 133 134 135 136 137 138 139 141 142 144 146 147 150 151 153 154 155
 156 157 159 160 161 162 163 164 165 166 167 168 169 170 172 173 174 175
 176 177 178 179 180 181 183 185 187 189 190 191 192 193 194 197 199 200
 202 203 204 205 206 208 209 210 213]
TEST: [  8  10  20  22  32  36  38  43  49  57  59  73  74  85  89 102 105 111
 116 118 120 123 124 127 140 143 145 148 149 152 158 171 182 184 186 188
 195 196 198 201 207 211 212]
Confusion matrix
[[ 7  4  0  0  0  0]
 [ 5 10  0  1  0  0]
 [ 2  2  0  0  0  0]
 [ 0  0  0  1  0  0]
 [ 0  0  0  0  

## StratifiedKFold (분류)

In [24]:
# KFold는 문제집 핀 곳만 푼 느낌이면 이건 무작위로 페이지 펴서 푼 느낌. ( train/test 분포를 보다 동일하게 분배해줌 ) 
# 회귀는 연속성의 데이터가 많아서 골고루의 개념이 없음

In [29]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=2022)  # y값을 각 fold안에 골고루 넣어주기 때문에 굳이 셔플 True 할 필요 없음
cm_list = []
f1_list = []
    
for train_idx, test_idx in skf.split(X, y):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]         # iloc : train 데이터셋에서 인덱스가 train_idx와 동일한 것만 추출
    X_test,  y_test  = X.iloc[test_idx], y.iloc[test_idx]           # iloc : test 데이터셋에서 인덱스가 test_idx와 동일한 것만 추출
    print("TRAIN:", train_idx)
    print("TEST:", test_idx)
    
    lr_model = LogisticRegression(penalty='l2', C=1.0)
    lr_model.fit(X_train, y_train)
    lr_pred = lr_model.predict(X_test)
    
    print('Confusion matrix')
    print(confusion_matrix(y_test, lr_pred))   
    cm_list.append(confusion_matrix(y_test, lr_pred)) 
    
    print(f"F1        : {f1_score(y_test, lr_pred, average='macro'):.4f}")
    f1_list.append(f1_score(y_test, lr_pred, average='macro'))
    
print('fl score : ',np.mean(f1_list))

TRAIN: [  0   2   3   6   7  10  11  12  13  16  17  18  19  20  21  23  24  25
  28  31  32  33  35  37  39  42  44  45  46  47  48  49  52  53  54  55
  56  57  60  61  62  63  65  67  68  69  70  71  72  73  75  77  78  79
  81  82  84  87  88  89  90  93  95  97  98  99 101 103 104 105 106 107
 108 110 114 115 116 117 120 121 123 124 125 126 127 130 132 133 134 137
 138 139 140 142 143 144 145 146 147 148 151 152 153 156 158 159 160 161
 164 165 166 167 170 171 172 173 175 176 178 180 181 182 184 186 187 188
 190 191 193 194 195 196 197 198 199 200 201 203 207 209 211 213]
TEST: [  1   4   5   8   9  14  15  22  26  27  29  30  34  36  38  40  41  43
  50  51  58  59  64  66  74  76  80  83  85  86  91  92  94  96 100 102
 109 111 112 113 118 119 122 128 129 131 135 136 141 149 150 154 155 157
 162 163 168 169 174 177 179 183 185 189 192 202 204 205 206 208 210 212]
Confusion matrix
[[18  6  0  0  0  0]
 [ 6 19  0  0  0  0]
 [ 5  1  0  0  0  0]
 [ 0  1  0  2  0  1]
 [ 1  0  0  0  2

## Cross_val_score

In [30]:
# 컴퓨터가 알아서 train/test 분리 후 교차검증해서 점수 출력
# 중간과정의 개입이 안들어가짐
# 비교, 가공 등 처리해야 될 것들이 있다면 다른 모델 사용 ( but 주로 Cross_val_score 사용 )

In [31]:
scores_arr = cross_val_score(lr_model , X, y , scoring="f1_macro", cv=3)
print(scores_arr)   # split 각각의 평가값
print(scores_arr.mean())  # split 각각 평가값의 평균

# 이걸 전부 cross_val_score가 다 알아서 처리해주고 결과만 리턴해줌 
# for train_index ,test_index  in skf.split(X  , y): 
#     train_X, test_X = X.iloc[train_index], X.iloc[test_index]
#     train_y , test_y = y.iloc[train_index], y.iloc[test_index]
#     print("TRAIN:", train_index, "TEST:", test_index)
#     print(train_X.shape, test_X.shape , train_y.shape , test_y.shape )
#     print(X_train.value_counts())
#     print(y_train.value_counts())

#     lr_model = LogisticRegression(penalty='l2', C=1.0)
#     lr_model.fit(X_train, y_train)
#     computer_pred  = lr_model.predict(X_test)
    
#     f1 = f1_score( y_test, computer_pred, average = 'macro')
#     f1_list.append(f1)
#     print(f1)
#     print(confusion_matrix(y_test, computer_pred))
      # return np.array(f1_list)

[0.39864147 0.55656667 0.50746123]
0.4875564559207806


# 모델교체 

<pre>
<b>CART Model (Classification And Regression Trees)</b>
  > 분류/회귀 둘 다 가능

<b>tree:</b>
    대표적인 모델 - decisiontreeclassifier
    단점 - 과적합 ( 데이터에 딱 맞춰서 학습 >> 새로운 유형의 데이터가 나오면 점수 나락) 
    
<b>bagging:</b>
    대표적인 모델 - randomforestclassifier

<b>boosting:</b> 
    대표적인 모델 - adaboostclassifier
                  - xgboost lightgbm catboost ( feature 가 너무 많아 과적함을 피함 )
    데이터 계속 복제 > 틀린 쪽 버리고 맞춘 쪽 가중치 세게줌 > 의미없는 데이터 버리고 좋은 데이터로 최종 fitting               
    가중치 조절로 선명하고 좋은 트리 만들어줌
    다양한 모델 사용한 효과
     
<b>votingclassifier:</b> 
    bagging 과 boosting 섞음  

In [33]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from  sklearn.ensemble import GradientBoostingClassifier

In [46]:
model_list = [ ("LR", LogisticRegression(penalty='l2', C=1.0)),
               ("DT", DecisionTreeClassifier()),
               ("RF", RandomForestClassifier()),   # DecisionTreeClassifier 100개 돌린 효과
               ("GB", GradientBoostingClassifier())]
for (name, model) in model_list:
    print (name)

LR
DT
RF
GB


# 하이퍼 파라미터

In [48]:
# 모델학습에 많은 영향을 미친다
# 파라미터 : 데이터로부터 추정된 조정불가한 값 
# 하이퍼 파라미터 : 모델링을 위해 설정해주는 값
# https://blog.naver.com/leehgyu/222388321634

## GridSearchCV

<pre>
# <b>파라미터를 손댐으로써 모델을 교체하지 않더라도 모델튜닝이 가능 </b> 

# <b>최상의 모델을 찾은 후 훈련 세트 전체를 사용해 최종 모델을 훈련 </b>


- <b>oob_score</b> : True 로 설정시 따로 test 줄 필요없음 (나머지 데이터로 알아서 처리)

- <b>gini(불순도)</b>    : 불순도 높은 데이터를 가지치기 함

- <b>entropy(정보량)</b> : 정보량 많은 쪽으로 이동


In [51]:
# GridSearchCV(estimator, param_grid, , scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2n_jobs', error_score=nan, return_train_score=False)

In [54]:
myprm = { 'criterion' : ['gini','entropy']  ,  'max_depth' : [7,9,11,13] ,  'min_samples_split': [1,2,3] }  # 불순도, 정보량 기준으로 12000번 학습해줘  

X = np.concatenate((X_train_scaler,X_test_scaler ),axis = 0  )
y = np.concatenate((y_train_scaler,y_test_scaler ),axis = 0  )

gcv_model = GridSearchCV(rf_model , param_grid=myprm , scoring = 'f1_macro' , refit = True , cv= 5)  #refit고정필수 ! 안하면 한 번 하고 끝냄  
gcv_model.fit(X , y ) # 학습해봐
print(gcv_model.best_params_)
print(gcv_model.best_score_)

NameError: name 'X_train_scaler' is not defined

## Random Forest

In [53]:
# RandomForestClassifier(n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0,
# max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0,
# warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)